In [358]:
from bs4 import BeautifulSoup as bs
from pprint import pprint
import json
import re
import requests
import pandas as pd
import re

In [328]:
pd.set_option('display.max_columns', None)
pd.options.mode.chained_assignment = None  # default='warn'

In [46]:
response = requests.get('https://rskrf.ru/ratings/produkty-pitaniya')
response

<Response [200]>

In [297]:
soup = bs(response.content, 'html.parser')

In [298]:
tag_li = soup.find_all('div', class_='h5 card-title with-text')

In [300]:
result = pd.DataFrame({"Наименование": [i.text for i in tag_li]})

In [368]:
test_df = pd.DataFrame(columns=['общий рейтинг', 'безопасность', 'качество'])
test_df
# циклом пробегаемся по всем продуктам и обрабатываем каждую ссылку

for link in tag_li:
    # создаём адрес продукта
    href = 'https://rskrf.ru/' + link.find('a').get('href')
    response_2 = requests.get(href)
    # создаём дом
    dom = bs(response_2.content, 'html.parser')
    # ищем позиции рейтинга
    tag_2 = dom.find_all('div', class_='rating-item')
    try:
        tag_3 = dom.find_all(string=re.compile('Пищевая ценность'))[1]
    except:
        pass
    indexes = []
    var_df = pd.DataFrame([{'общий рейтинг': '', 'безопасность': '', 'качество': '', 'ценность': ''}])
    if tag_3:
        var_df['ценность'].values[0] = tag_3
    else:
        pass
    for count,item in enumerate(tag_2):
        if count == 13:
            break
        else:
            pos = item.find_all('span')[0].text.lower().strip()
            if pos in ['общий рейтинг', 'безопасность', 'качество']:
                var_df[pos].values[0] = item.find_all('span')[1]
            else:
                pass
    test_df = test_df.append(var_df, ignore_index = True )
test_df



C:\Users\a.shernazarov\AppData\Local\Temp\ipykernel_11792\2603027064.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_df = test_df.append(var_df, ignore_index = True )
C:\Users\a.shernazarov\AppData\Local\Temp\ipykernel_11792\2603027064.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_df = test_df.append(var_df, ignore_index = True )
C:\Users\a.shernazarov\AppData\Local\Temp\ipykernel_11792\2603027064.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_df = test_df.append(var_df, ignore_index = True )
C:\Users\a.shernazarov\AppData\Local\Temp\ipykernel_11792\2603027064.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat in

,общий рейтинг,безопасность,качество,ценность
0,[5.0000],[5.000],[5.000],"Пищевая ценность 100 г продукта: жир от 3,6 г ..."
1,[5.0000],[5.000],[5.000],Пищевая ценность продукта в 100 г: жира - от 3...
2,[5.0000],[5.000],[5.000],\nПищевая ценность соответствует установленным...
3,[5.0000],[5.000],[5.000],Пищевая ценность (содержание в 100 г продукта)...
4,[5.0000],[5.000],,\nПищевая ценность соответствует установленным...
5,[5.0000],[5.000],,\nПищевая ценность соответствует установленным...
6,[5.0000],[5.000],,\nПищевая ценность соответствует установленным...
7,[5.0000],[5.000],[5.000],Пищевая ценность продукта (средние значения) н...
8,[5.0000],[5.000],[5.000],Пищевая ценность в 100г продукта (средние знач...
9,[5.0000],[5.000],[5.000],"Пищевая ценность в 100 г продукта: жиры - 3,2 ..."


In [371]:
result_df = pd.concat([result, test_df], sort=False, axis=1)

In [372]:
result_df

,Наименование,общий рейтинг,безопасность,качество,ценность
0,Братья Чебурашкины Молоко питьевое пастеризова...,[5.0000],[5.000],[5.000],"Пищевая ценность 100 г продукта: жир от 3,6 г ..."
1,Му-у Молоко цельное отборное питьевое пастериз...,[5.0000],[5.000],[5.000],Пищевая ценность продукта в 100 г: жира - от 3...
2,Снежок Сметана 20% «Снежок»,[5.0000],[5.000],[5.000],\nПищевая ценность соответствует установленным...
3,Молоко Оренбуржья Молоко питьевое пастеризован...,[5.0000],[5.000],[5.000],Пищевая ценность (содержание в 100 г продукта)...
4,"Коровка из Кореновки Сметана, массовая доля жи...",[5.0000],[5.000],,\nПищевая ценность соответствует установленным...
5,Агрокомплекс Выселковский Сметана с массовой д...,[5.0000],[5.000],,\nПищевая ценность соответствует установленным...
6,"Лебедянь молоко Сметана, массовая доля жира 20...",[5.0000],[5.000],,\nПищевая ценность соответствует установленным...
7,Любимое молоко Молоко питьевое пастеризованное...,[5.0000],[5.000],[5.000],Пищевая ценность продукта (средние значения) н...
8,Молоко Брянского молочного комбината Молоко пи...,[5.0000],[5.000],[5.000],Пищевая ценность в 100г продукта (средние знач...
9,Сударыня Молоко питьевое ультрапастеризованн...,[5.0000],[5.000],[5.000],"Пищевая ценность в 100 г продукта: жиры - 3,2 ..."
